In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Загрузка данных.

In [20]:
# Датасет Answers.csv
answers = pd.read_csv('../Data/Common/Answers.csv', encoding='latin1')
# Выводим первые 5 строк
answers.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."


In [21]:
# Датасет Questions.csv
questions = pd.read_csv('../Data/Common/Questions.csv', encoding='latin1')
# Выводим первые 5 строк
questions.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [22]:
# Датасет Tags.csv
tags = pd.read_csv('../Data/Common/Tags.csv', encoding='latin1')
# Выводим первые 5 строк
tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [85]:
import re
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')  # Загрузка стоп-слов
nltk.download('wordnet')  # Загрузка WordNet
nltk.download('averaged_perceptron_tagger_eng')  # Загрузка тегов

[nltk_data] Downloading package stopwords to C:\Users\VORANDPAV BIG
[nltk_data]     SPB\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\VORANDPAV BIG
[nltk_data]     SPB\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\VORANDPAV BIG
[nltk_data]     SPB\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.


True

In [113]:
stop_words = set(stopwords.words('english'))  # Загрузка стоп-слов
contractions = ['ive', 'youre', 'theyre', 'hes', 'shes', 'its', 'weve', 'theyve', 'im', 'isnt', 'wasnt', 'werent',
                'hasnt', 'havent', 'dont', 'doesnt', 'didnt', 'cant', 'couldnt', 'shouldnt', 'mightnt', 'mustnt',
                'wouldnt']  # Список сокращений
lemmatizer = WordNetLemmatizer()  # Загрузка лемматизатора


# Функция опеределения части речи для лемматизации
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)


# Функция для очистки текста
def clean_text(text):
    if not isinstance(text, str):  # Проверяем, что текст является строкой
        return ''

    text = re.sub(r'<code>.*?</code>', '', text, flags=re.DOTALL)  # Удаляем блоки <code>
    text = re.sub(r'\`[^\`]*\`', '', text)  # Удаляем инлайн-код в Markdown
    text = re.sub(r'<[^>]*>', '', text)  # Удаляем оставшиеся HTML-теги
    text = re.sub(r'\[.*?\]\(.*?\)', '', text)  # Удаляем Markdown-ссылки
    text = re.sub(r'"[^"]*"', '', text)  # Удаляем текст в кавычках
    text = re.sub(r'\s+', ' ', text).strip()  # Удаляем лишние пробелы

    text = text.lower()  # Приводим текст к нижнему регистру
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Удаляем все кроме букв, цифр и пробелов

    words = text.split()  # Разбиваем текст на слова
    words = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in words if
             word not in stop_words and word not in contractions]  # Удаляем стоп-слова и сокращения, лемматизируем слова
    text = ' '.join(words)  # Объединяем слова обратно в текст

    return text


# Проверяем, что функция работает
print('Before cleaning:')
print(questions['Body'].iloc[0])
print('After cleaning:')
print(clean_text((questions['Body'].iloc[0])))
print(get_wordnet_pos('running'))
print(get_wordnet_pos('exists'))
print(get_wordnet_pos('exist'))

Before cleaning:
<p>I've written a database generation script in <a href="http://en.wikipedia.org/wiki/SQL">SQL</a> and want to execute it in my <a href="http://en.wikipedia.org/wiki/Adobe_Integrated_Runtime">Adobe AIR</a> application:</p>

<pre><code>Create Table tRole (
      roleID integer Primary Key
      ,roleName varchar(40)
);
Create Table tFile (
    fileID integer Primary Key
    ,fileName varchar(50)
    ,fileDescription varchar(500)
    ,thumbnailID integer
    ,fileFormatID integer
    ,categoryID integer
    ,isFavorite boolean
    ,dateAdded date
    ,globalAccessCount integer
    ,lastAccessTime date
    ,downloadComplete boolean
    ,isNew boolean
    ,isSpotlight boolean
    ,duration varchar(30)
);
Create Table tCategory (
    categoryID integer Primary Key
    ,categoryName varchar(50)
    ,parent_categoryID integer
);
...
</code></pre>

<p>I execute this in Adobe AIR using the following methods:</p>

<pre><code>public static function RunSqlFromFile(fileName:String)

In [103]:
# Применяем функцию к Body вопросов и ответов
questions_clear = questions.copy()
answers_clear = answers.copy()
questions_clear['Body'] = questions_clear['Body'].swifter.apply(clean_text)
answers_clear['Body'] = answers_clear['Body'].swifter.apply(clean_text)

Pandas Apply: 100%|██████████| 2014516/2014516 [41:50<00:00, 802.46it/s] 


In [166]:
# Сохраняем очищенные вопросы и ответы
questions_clear.to_csv('../Data/Processed/questions.csv', index=False, encoding='latin1')
answers_clear.to_csv('../Data/Processed/questions.csv', index=False, encoding='latin1')

In [106]:
# Проверяем, что функция сработала
print(questions_clear['Body'].head())
print(answers_clear['Body'].head())

0    write database generation script sql want exec...
1    really good tutorial explain branching merge a...
2    anyone get experience create sqlbased aspnet s...
3    something pseudosolved many time never quite f...
4    little game write c us database backend tradin...
Name: Body, dtype: object
0    version control subversion good resource sourc...
1    wound use kind hack actually work pretty well ...
2    read somewhere human eye distinguish less 4 va...
3    yes thought soon figure another domainspecific...
4    oleg shilos c script solution code project rea...
Name: Body, dtype: object
